In [2]:
import requests
import pandas as pd
import datetime
from datetime import timedelta
from datetime import timezone
import time

In [3]:
#pool = '0x39AA39c021dfbaE8faC545936693aC917d5E7563' #cUSDC
#pool = '0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9' #cUSDT
pool = '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643' #cDAi
#pool = '0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5' #cETH
#pool = 'xc11b1268c1a384e55c48c2391d8d480264a3a7f4' #cWBTC - not
#pool = '0x35A18000230DA775CAc24873d00Ff85BccdeD550' #cUNI
#pool = '0xFAce851a4921ce59e912d19329929CE6da6EB0c7' #cLINK

asset = 'dai'



date_1 = int(datetime.datetime(2022, 9, 29, 0,0).replace(tzinfo=timezone.utc).timestamp())
date_2 = int(datetime.datetime(2022, 9, 30, 0,0).replace(tzinfo=timezone.utc).timestamp())

In [4]:
lst = []

In [5]:
def process_compound(raw):
    epoch = raw['supply_rates'][0]['block_timestamp']
    time = datetime.datetime.utcfromtimestamp(epoch)
    supply = raw['supply_rates'][0]['rate']
    borrow = raw['borrow_rates'][0]['rate']
        
    lst.append(['', epoch, time, supply, borrow])

In [6]:
cont = True

while cont:
    query = f'https://api.compound.finance/api/v2/market_history/graph?asset={pool}&num_buckets=1&min_block_timestamp={date_1}&max_block_timestamp={date_2}'
    response = requests.get(query)
    raw = response.json()
    process_compound(raw)
    date_1 = date_1 - 86400
    date_2 = date_2 - 86400
    print(datetime.datetime.utcfromtimestamp(date_2).strftime('%d-%m-%Y'))
    time.sleep(3)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

temp = pd.read_csv('data/compound_usdt_interest.csv')
df_new = pd.concat([temp, df[df['timestamp'] >= '2022-08-31']])
df = df_new

In [ ]:
df_compound = pd.DataFrame(lst, columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow'])
df_compound['asset'] = asset
df = df_compound.copy()
df = df.sort_values(by='timestamp', ascending=True)
df = df.reset_index(drop=True)
df

In [ ]:
#store in csv
df.to_csv(f'data/compound_{asset}_interest.csv', index=False)